In [14]:
import torch
import torch.nn as nn 
import torch.nn.functional as F 
import torchvision 
import torchvision.transforms as transforms

import matplotlib.pyplot as plt 
%matplotlib inline

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [15]:
transform = transforms.Compose(
    [transforms.ToTensor(), 
     transforms.Resize((227, 227)),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


# already downloaded
train = torchvision.datasets.CIFAR10(root='./data/train_CIFAR10', train=True,
                                    download=False, transform=transform)

train_loader = torch.utils.data.DataLoader(train, batch_size=64,
                                          shuffle=True, num_workers=2)
# already downloaded 
test = torchvision.datasets.CIFAR10(root='./data/test_CIFAR10', train=False,
                                    download=False, transform=transform)

test_loader = torch.utils.data.DataLoader(train, batch_size=64,
                                          shuffle=True, num_workers=2)

In [93]:
class Inception(nn.Module):
    def __init__(self, in_channels, out_1, red_3, out_3, red_5, out_5, out_pool):
        super(Inception, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels, out_1, kernel_size=1) 
        
        self.conv3 = nn.Sequential( 
            nn.Conv2d(in_channels, red_3, kernel_size=1, padding=0),
            nn.Conv2d(red_3, out_3, kernel_size=3, padding=1))
        
        self.conv5 = nn.Sequential(
            nn.Conv2d(in_channels, red_5, kernel_size=1), 
            nn.Conv2d(red_5, out_5, kernel_size=5, padding=2)) 
        
        self.max = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, padding=1, stride=1), 
            nn.Conv2d(in_channels, out_pool, kernel_size=1))
        
    def forward(self, x): 
        x1 = self.conv1(x)
        x2 = self.conv3(x)
        x3 = self.conv5(x)
        x4 = self.max(x)
        out = torch.cat([x1, x2, x3, x4], dim=1)
        return out
        
        return out
    
class GooglLeNet(nn.Module): 
    def __init__(self, out_channels=10):
        super().__init__()
        
        self.l1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3), 
            nn.MaxPool2d(kernel_size=3, stride=2))
             
        self.l2 = nn.Sequential(
            nn.Conv2d(64, 192, kernel_size=3, stride=1, padding=1), 
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
        
        
        self.inc3a = Inception(192, 64, 96, 128, 16, 32, 32)
        self.inc3b = Inception(256, 128, 128, 192, 32, 96, 64)
        self.max3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
            
        self.l4 = nn.Sequential(
            Inception(480, 192, 96, 208, 16, 48, 64), 
            Inception(512, 160, 112, 224, 24, 64, 64), 
            Inception(512, 128, 128, 256, 24, 64, 64), 
            Inception(512, 112, 144, 288, 32, 64, 64), 
            Inception(528, 256, 160, 320, 32, 128, 128),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)) 
    
        
        self.inc5a = Inception(832, 256, 160, 320, 32, 128, 128)
        self.inc5b = Inception(832, 384, 192, 384, 48, 128, 128) 
        self.avg = nn.AvgPool2d(kernel_size=7, stride=1)
        
        self.drop = nn.Dropout(p=0.4)
            
        self.linear = nn.Linear(1024,out_channels)
    
    def forward(self, x): 
        x = self.l1(x) 
        x = self.l2(x)
        x = self.inc3a(x) 
        x = self.inc3b(x) 
        x = self.max3(x) 
        x = self.l4(x)
        x = self.inc5a(x) 
        print(x.shape)
        x = self.inc5b(x) 
        x = self.avg(x) 
        x = self.drop(x) 
        x = x.reshape(x.shape[0], -1)
        x = self.linear(x) 
        
        return x


In [17]:
# class NaiveInception(nn.Module):
#     '''
#     Computer can likely only tolerate Naive inception model
#     '''
#     def __init__(self, in_channels, out_1, out_3, out_5):
#         super().__init__()
        
#         self.conv1 = nn.Conv2d(in_channels, out_1, kernel_size=1, stride=1, padding=0)
#         self.conv2 = nn.Conv2d(in_channels, out_3, kernel_size=3, stride=1, padding=1)
#         self.conv3 = nn.Conv2d(in_channels, out_5, kernel_size=5, stride=1, padding=2)            
#         self.maxpool = nn.MaxPool2d(kernel_size=3, stride=1, padding=1) 
        
#     def forward(self, x):
#         x1 = self.conv1(x)
#         x2 = self.conv2(x)
#         x3 = self.conv3(x)
#         x4 = self.maxpool(x)
#         out = torch.cat([x1, x2, x3, x4], dim=1)
#         return out
    
    
    
# class GooglLeNetNaive(nn.Module): 
#     def __init__(self, out_channels):
#         super().__init__()
        
#         self.l1 = nn.Sequential(
#             nn.Conv2d(3, 64, kernel_size=7, stride=2), 
#             nn.MaxPool2d(kernel_size=3, stride=2))
             
#         self.l2 = nn.Sequential(
#             nn.Conv2d(64, 192, kernel_size=3, stride=1), 
#             nn.MaxPool2d(kernel_size=3, stride=1))
        
#         self.l3 = nn.Sequential(
#             NaiveInception(192, 256),
#             NaiveInception(256, 480), 
#             nn.MaxPool2d(kernel_size=3, stride=2))
            
#         self.l4 = nn.Sequential(
#             NaiveInception(512, 512), 
#             NaiveInception(512, 512), 
#             NaiveInception(512, 528), 
#             NaiveInception(528, 832), 
#             nn.MaxPool2d(kernel_size=3, stride=2)) 
    
#         self.l5 = nn.Sequential(
#             NaiveInception(832, 1024), 
#             NaiveInception(1024, 1024), 
#             nn.AvgPool2d(kernel_size=7, stride=1), 
#             nn.Dropout(p=0.4))
            
#         self.linear = nn.Linear(1000,1000)
    
#     def forward(self, x): 
#         out = self.l1(x) 
#         out = self.l2(out)
#         out = self.l3(out) 
#         out = self.l4(out) 
#         out = self.l5(out) 
#         out = self.linear(out) 
#         out = nn.Softmax()
        
#         return out

In [94]:
# training loop 
torch.manual_seed(423984) 
googlenet = GooglLeNet()
googlenet.to(device)

loss_fn = nn.CrossEntropyLoss()
# optimizer parameters according to paper
optimizer = torch.optim.SGD(googlenet.parameters(),lr=0.01, momentum=0.9, weight_decay=0.0005)

In [95]:
from tqdm.auto import tqdm 

torch.manual_seed(424)

epochs = 5

for epoch in tqdm(range(epochs)):
    print(f'Epoch: {epoch}\n-----')
    
    train_loss = 0
    
    for batch, (x, y) in enumerate(train_loader): 
        x = x.to(device)
        
        y_pred = googlenet(x)
        
        loss = loss_fn(y_pred, y)
        train_loss += loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    train_loss /= len(train)
    
    test_loss = 0
    
    with torch.inference_mode(): 
        for x, y in test:
            test_pred = googlenet(x) 
            
            loss = loss_fn(test_pred, y)
            test_loss += loss
            
        test_loss /= len(test)
    
    print(f"\nTrain loss: {train_loss:.5f} | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%\n")


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0
-----


/Users/finhardy/opt/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/finhardy/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  Referenced from: <F30FD7F2-B214-3D4A-93DD-0D484FBE6931> /Users/finhardy/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <9D4C7FD2-49A8-383A-AC3E-A560DE81B0D8> /Users/finhardy/opt/anaconda3/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")
/Users/finhardy/opt/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/finhardy/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not foun

input size: torch.Size([64, 3, 227, 227])
Output from Conv: torch.Size([64, 64, 56, 56])
Output from Conv: torch.Size([64, 192, 28, 28])
Shape after inception 3a: torch.Size([64, 256, 28, 28])
Shape after inception 3b: torch.Size([64, 480, 28, 28])
Output from inception 3a + 3b: torch.Size([64, 480, 14, 14])
Output from inception 4a-4d: torch.Size([64, 832, 7, 7])
torch.Size([64, 832, 7, 7])
input size: torch.Size([64, 3, 227, 227])
Output from Conv: torch.Size([64, 64, 56, 56])
Output from Conv: torch.Size([64, 192, 28, 28])
Shape after inception 3a: torch.Size([64, 256, 28, 28])
Shape after inception 3b: torch.Size([64, 480, 28, 28])
Output from inception 3a + 3b: torch.Size([64, 480, 14, 14])
Output from inception 4a-4d: torch.Size([64, 832, 7, 7])
torch.Size([64, 832, 7, 7])
input size: torch.Size([64, 3, 227, 227])
Output from Conv: torch.Size([64, 64, 56, 56])
Output from Conv: torch.Size([64, 192, 28, 28])
Shape after inception 3a: torch.Size([64, 256, 28, 28])
Shape after incep

KeyboardInterrupt: 